# Analyse predatory journals

## Vraag

1. Welke journals in Pure komen ook voor in lijst Bealls website?
2. Welke journals in Pure komen ook voor in predatory journals R-pakket?
3. Welke journals in Pure worden vals-positief gematched met lijst uit R-pakket?

### Pure - Bealls
De lijst met journals is gebaseerd op volgend GitHub project

**pred_journals_github_jName.tsv**

### Pure - R pakket
We zijn gestart met de lijst van journals uit het R-pakket ["predatory"](https://cran.r-project.org/web/packages/predatory/predatory.pdf).
De bijgevoegde [csv](./data/rpackage_predpub.csv) bevat 1136 journals.

In [2]:
import pandas as pd
import numpy as np

journals = './data/rpackage_predpub.csv'

data = pd.read_csv(journals)
#data.info()
data.head(3)

,issn,name,main.url,type,ind.url
0,1459-0255,WFL Publisher,http://world-food.net/,publisher,http://world-food.net/category/journals/
1,0195-9131,International Agency for Development of Cultur...,http://iadces.com/,publisher,https://iadces.com/
2,1452-3981,International Journal of Electrochemical Science,http://www.electrochemsci.org/,ind_journal,http://www.electrochemsci.org/


Op basis van de issn's in deze lijst hebben we een lucene query gemaakt en in Pure uitgevoerd.
<br/>We vinden een match voor 100 journals.

In [78]:
journalMatches = './data/Pure_matches_issn_v2.csv'

d = pd.read_table(journalMatches, sep=';')
#d.info()
d.head(3)

,1.1 Titles[1],1.2 Titles[2],2.1 ISSNs[1],2.2 ISSNs[2],3.1 Additional searchable ISSN (Electronic)[1],5 Id
0,Hepatobiliary Surgery and Nutrition,NaN,2304-3881,NaN,2304-389X,4832862
1,Modern Economy,NaN,2152-7245,NaN,2152-7261,35730169
2,Advances in Anthropology,NaN,2163-9353,NaN,2163-9361,30907716


### Vals-positieve ISSN's
Een aantal gematchte journals leken vals positieven te zijn.<br/>
Om dit te controleren vergelijken we de lijst van 100 journals met de journals uit [Journal Citation Index](https://jcr.incites.thomsonreuters.com/) en [DOAJ](https://doaj.org/).

De DOAJ dataset bevat *7850* issn's waar naar gekeken wordt, de JCR dataset *14199*.

In [79]:
doaj = './data/DOAJ.csv'
d = pd.read_csv(doaj, sep='\t', header=None, encoding='ansi', names=['journal', 'issn'])
#d.info()
d.head(3)

,journal,issn
0,Oklahoma Native Plant Record,2379-4011
1,SPAFA Journal,2586-8721
2,Southeast Asian Review of English,0127-046X


In [80]:
jcr = './data/JCR_SCIE_2017.csv'
d = pd.read_csv(jcr, sep=';')
#d.info()
d.head(3)

,Title20,ISO_ABBREV,TITLE,ISSN
0,J MED SYST,J. Med. Syst.,JOURNAL OF MEDICAL SYSTEMS,0148-5598
1,J MEMBRANE SCI,J. Membr. Sci.,JOURNAL OF MEMBRANE SCIENCE,0376-7388
2,J METAMORPH GEOL,J. Metamorph. Geol.,JOURNAL OF METAMORPHIC GEOLOGY,0263-4929


## Analyse na whitelist
Met deze drie datasets kunnen we nu van start:
1. matches in Pure obv issn
2. JCR referentie
3. DOAJ referentie

In OpenRefine laden we de drie datasets in. In de 1e dataset (Pure_matches_issn) creëeren we extra kolommen waar we nagaan of het ISSN ook voorkomt in de twee referentie datasets.
`ISSN kolom - Edit column - Add column based on this column...`
```
cross(value,'DOAJ','ISSN').length()
```
Zo vergelijken we het Pure issn `value` met de kolom `ISSN` uit dataset `DOAJ` (en daarna JCR); dit is een array waar we vervolgens de `length()` van bepalen om te kijken of we een match hebben.<br/>
<br/>Onderstaande tabel toont het aantal matches voor het print-issn (1) en het e-issn (2) bij zowel JCR als DOAJ.

|       | JCR 1 | DOAJ 1 | JCR 2 | DOAJ 2 |
| ----- | ----- | ------ | ----- | ------ |
| **0** | 49    | 98     | 81    | 100    |
| **1** | 24    | 2      | 1     |        |
| **2** | 20    |        | 2     |        |
| **3** | 5     |        |       |        |
| **4** | 2     |        |       |        |

Bij journals zonder match kunnen we de grootste waarschijnlijkheid van 'predatory' te zijn, toeschrijven.<br/>
In totaal gaat dit over **45** journals die geen enkele match hebben.

In [76]:
journals = './data/Journals_DOAJ_JCR.tsv'
d = pd.read_csv(journals, sep='\t',)

nomatch = d[(d.jcr == 0) & (d.doaj == 0) & (d.jcr2 != 1) & (d.jcr != 2)]
nomatch

,Title,1.2 Titles[2],2.1 ISSNs[1],2.2 ISSNs[2],3.1 Additional searchable ISSN (Electronic)[1],jcr,jcr2,doaj,doaj2
1,Modern Economy,NaN,2152-7245,NaN,2152-7261,0,0.0,0,0.0
2,Advances in Anthropology,NaN,2163-9353,NaN,2163-9361,0,0.0,0,0.0
3,"World Academy of Science, Engineering and Tech...",NaN,2010-376X,NaN,2010-3778,0,0.0,0,0.0
4,American Journal of Industrial and Business Ma...,NaN,2164-5167,NaN,2164-5175,0,0.0,0,0.0
5,International Journal of Geosciences,International Journal of Geosciences (IJG),2156-8359,NaN,2156-8367,0,0.0,0,0.0
6,Journal of Arts and Humanities,JAH,2167-9045,NaN,2167-9053,0,0.0,0,0.0
7,International Journal of Design & Nature and E...,NaN,1755-7437,NaN,1755-7445,0,0.0,0,0.0
8,Energy and Power Engineering,NaN,1947-3818,1949-243X,1947-3818,0,0.0,0,0.0
9,International Journal of Humanities and Social...,NaN,2220-8488,NaN,2221-0989,0,0.0,0,0.0
11,International Journal of Renewable Energy Rese...,NaN,1309-0127,NaN,1309-0127,0,0.0,0,0.0


## Analyse 'vals positieven'
Een groot aandeel journals werd in eerste instantie gematcht obv. ISSN, maar komen toch voor in DOAJ, JCR of beiden.<br/>
Hieronder bekijken we verder de 'vals positieven'.

In [13]:
fp = './data/False_Positives.csv'
d = pd.read_csv(fp, sep=';')
#d.info()
falsePositives = d[(d.Link == 0)]
falsePositives

,Title,pISSN,eISSN,jcr,jcr2,doaj,doaj2,Reden,Link
1,Journal of Vibration and Acoustics,1048-9002,1528-8927,3,0,0,0,IADCES,0
2,British Journal of Neurosurgery,0268-8697,1360-046X,2,0,0,0,IADCES,0
3,IEEE Transactions on Information Forensics and...,1556-6013,NaN,2,/,0,/,Geen link gevonden,0
4,Journal of Energy Engineering,0733-9402,NaN,2,/,0,/,IADCES,0
5,EURASIP Journal on Wireless Communications and...,1687-1499,1687-1499,2,2,0,0,Geen link gevonden,0
6,Obesity Reviews,1467-7881,1467-789X,1,0,0,0,Geen link gevonden,0
8,Journal of Mechanisms and Robotics,1942-4302,1942-4310,2,0,0,0,IADCES,0
9,Obesity,1930-7381,1930-739X,2,0,0,0,Geen link gevonden,0
10,Biotechnology Journal,1860-6768,1860-7314,2,0,0,0,IADCES,0
11,Freshwater Science,2161-9549,2161-9565,2,0,0,0,IADCES,0


Op een totaal van 51 journals vinden we 47 vals positieven.<br/>
Opmerking: bij IADCES is er wel een link te vinden. IADCES lijkt een [predatory organisatie](http://leiterlawschool.typepad.com/leiter/2017/03/watch-out-for-the-international-agency-for-development-of-culture-education-and-science-iadces.html) te zijn die onderzoekers tegen betaling zou helpen bij het publiceren in (o.a.) deze gekoppelde journals. Op zich zijn deze journals zelf dus **niet** betrokken.

In [10]:
positives = d[(d.Link == 1)]
positives

,Title,pISSN,eISSN,jcr,jcr2,doaj,doaj2,Reden,Link
0,Hepatobiliary Surgery and Nutrition,2304-3881,2304-389X,3,0,0,0,Publisher (AME) op lijst Bealls,1
7,Frontiers in Bioscience (Landmark edition),1093-9946,1093-4715,2,0,0,0,Publisher (Frontiers in Bioscience) op lijst B...,1
13,Journal of Thoracic Disease,2072-1439,2077-6624,1,0,0,0,Publisher (AME) op lijst Bealls,1
47,Wireless Personal Communications,0929-6212,1572-834X,1,0,0,0,"E-issn vroeger gelinkt aan Gyancity, nu onder ...",1


Voor vier journals is er **wel** een link gevonden, terwijl ze ook in JCR opgenomen zijn. Het gaat om twee journals gelinkt aan de uitgever *AME*, een aan *Frontiers in Bioscience* en een aan *Gyancity*.